<a href="https://colab.research.google.com/github/saravanamuthu1/kryol_language_segementation/blob/main/Kryol_segementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional 
from tensorflow.keras.metrics import CategoricalAccuracy
from tqdm import tqdm
import csv

In [64]:
data = open('train.tsv', 'r')
all_words = []
all_segemented_words = []
for line in data:
    pieces = line.rstrip("\n").split("\t")
    all_words.append(pieces[0])
    all_segemented_words.append(pieces[1])
    

In [65]:
complete_list=[]
all_label = []
for i in all_segemented_words:
  value=i.split("-")
  cur_word = []
  word_label=[]
  for i in range(0, len(value)):
        for j in range(0, len(value[i])):
            if j == 0:
                cur_word.append(value[i][j])
                word_label.append("B")
            else:
                cur_word.append(value[i][j])
                word_label.append("I")
  complete_list.append(cur_word)
  all_label.append(word_label)


In [102]:
complete_list[3]

['v', 'e', 'j', 'e', 't', 'a', 'l']

In [99]:
len(complete_list)

12812

In [66]:
longest_string = max(complete_list, key=len)
print(len(longest_string))

18


In [100]:
len(all_label)

12812

In [67]:
max_len = 20

In [68]:
len_letters= len(set(letter for word in complete_list for letter in word ))

In [69]:
letters= list(set(letter for word in complete_list for letter in word ))
letters.append('<PAD>')

In [70]:
letters

['I',
 'f',
 'b',
 'R',
 'h',
 'P',
 'Y',
 'l',
 'i',
 'è',
 'T',
 'm',
 'C',
 's',
 'e',
 'Z',
 'v',
 'o',
 'ò',
 'A',
 'S',
 'c',
 'J',
 'N',
 'E',
 'O',
 'V',
 'à',
 'Ò',
 'W',
 'K',
 'g',
 'B',
 'L',
 'F',
 'j',
 'u',
 't',
 'D',
 'È',
 'M',
 'G',
 'r',
 'd',
 'k',
 'w',
 'p',
 'a',
 'y',
 'z',
 'n',
 '<PAD>']

In [71]:
tags_value= len(set(tag for tag_list in all_label for tag in tag_list))

In [72]:
tags_value= list(set(tag for tag_list in all_label for tag in tag_list))

In [73]:
tags_value.append('<PAD>')

In [74]:
word2index = {w: i for i, w in enumerate(letters)}

In [75]:
onehot = [[word2index[w] for w in s] for s in complete_list]

In [76]:
X = pad_sequences(maxlen=max_len, sequences=onehot, padding="post", value=len(letters)-1)

In [77]:
from tensorflow.keras.utils import to_categorical

In [78]:
tag2index = {t: i for i, t in enumerate(tags_value)}

In [79]:
onehot_y = [[tag2index[w] for w in s] for s in all_label]
y = pad_sequences(maxlen=max_len, sequences=onehot_y, padding="post", value=tag2index["<PAD>"])
y = to_categorical(y, num_classes=len(tags_value))

In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [81]:
model = Sequential()
model.add(Embedding(input_dim=len(letters), output_dim=50, input_length=max_len))
model.add(Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1)))
model.add(TimeDistributed(Dense(len(tags_value), activation="softmax")))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 50)            2600      
                                                                 
 bidirectional_2 (Bidirectio  (None, 20, 200)          120800    
 nal)                                                            
                                                                 
 time_distributed_1 (TimeDis  (None, 20, 3)            603       
 tributed)                                                       
                                                                 
Total params: 124,003
Trainable params: 124,003
Non-trainable params: 0
_________________________________________________________________
None


In [82]:
# run fit on Colab or hopper
history = model.fit(X_train, y_train, batch_size=5, epochs=1, validation_split=0.2, verbose=1)

1435/1435 [==============================] - 94s 62ms/step - loss: 0.0442 - accuracy: 0.9855 - val_loss: 0.0047 - val_accuracy: 0.9992


In [83]:
predicted=[]
acutal=[]

In [84]:
for test_sent in range(len(X_test)):
  pred = model.predict(np.array([X_test[test_sent]]))
  p = np.argmax(pred, axis=-1)
  p_actual = np.argmax(y_test[test_sent],axis=-1)
  for i,ix in enumerate(X_test[test_sent]):
    if ix==len(letters)-1:
        break
    predicted.append(tags_value[p[0][i]])
    acutal.append(tags_value[p_actual[i]])

In [85]:
df=pd.DataFrame(predicted)

In [86]:
df.value_counts()

B    23971
I     2324
dtype: int64

In [87]:
df1=pd.DataFrame(acutal)

In [88]:
df1.value_counts()

B    23965
I     2330
dtype: int64

In [89]:
count=0

In [90]:
total_value=list(zip(predicted,acutal))

In [91]:
from sklearn.metrics import accuracy_score


In [92]:
accuracy_score(predicted,acutal)

0.9978703175508652

In [93]:
true_pos=0
false_neg=0

In [94]:
for i,j in zip(predicted,acutal):
   if i == 'I' and j == 'I':
     true_pos += 1
   if i == 'B' and j == 'I':
      false_neg += 1


In [95]:
recall = 100 * (true_pos / (true_pos + false_neg))

In [96]:
true_pos1 = 0
false_pos1 = 0
for i,j in zip(predicted,acutal):
   if i == 'I' and j == 'I':
     true_pos1 += 1
   if i == 'I' and j == 'B':
      false_pos1 += 1
precision = 100 * (true_pos1 / (true_pos1 + false_pos1))

In [97]:
precision

98.92426850258175

In [98]:
fscore = (2 * (precision * recall)) / (precision + recall)
print("Fscore: ", fscore)

Fscore:  98.79673399226473
